## IMPORT DATA


In [2]:
import os
import re
import pandas
from IPython.display import HTML

input_filepath = "./data/T7_2025.csv"

df = pandas.read_csv(input_filepath)

HTML(df.head().to_html())

,Full name,Email address,Username,Course short name,Session start,Session duration
0,Aayushi Biswas,biswasaayushi77@gmail.com,biswasaayushi77,ETOP Beginner,"Tuesday, 15 July 2025, 3:54 PM",1 min 4 secs
1,Aayushi Biswas,biswasaayushi77@gmail.com,biswasaayushi77,ETOP Beginner,"Thursday, 17 July 2025, 8:02 PM",4 mins 6 secs
2,AG20P1095 Đỗ Thị Ngọc Trinh,thaotrinh0828@gmail.com,dotrinh020220,ETOP Beginner,"Wednesday, 16 July 2025, 6:57 PM",1 min 17 secs
3,AG20P1095 Đỗ Thị Ngọc Trinh,thaotrinh0828@gmail.com,dotrinh020220,Pronunciation - Beginner,"Saturday, 19 July 2025, 10:07 AM",1 min 25 secs
4,AG20P1095 Đỗ Thị Ngọc Trinh,thaotrinh0828@gmail.com,dotrinh020220,ETOP Beginner,"Saturday, 19 July 2025, 10:07 AM",1 min 18 secs


## FORMAT DATA


### SPLIT PALS_ID VS FULLNAME


In [3]:
# Tách cột fullname thành PALS_ID và FULLNAME mới
df[["PALS_ID", "FULLNAME"]] = df["Full name"].str.extract(r"(\S+)\s+(.+)")
df.drop(columns=["Full name"], inplace=True)

df.rename(
    columns={
        "PALS_ID": "PALS_ID",
        "FULLNAME": "FULLNAME",
        "Email address": "EMAIL",
        "Course short name": "COURSE",
        "Session start": "START_TIME",
        "Session duration": "DURATION",
    },
    inplace=True,
)

order = ["PALS_ID", "FULLNAME", "EMAIL", "COURSE", "START_TIME", "DURATION"]
df = df.reindex(columns=order)

HTML(df.head().to_html())

,PALS_ID,FULLNAME,EMAIL,COURSE,START_TIME,DURATION
0,Aayushi,Biswas,biswasaayushi77@gmail.com,ETOP Beginner,"Tuesday, 15 July 2025, 3:54 PM",1 min 4 secs
1,Aayushi,Biswas,biswasaayushi77@gmail.com,ETOP Beginner,"Thursday, 17 July 2025, 8:02 PM",4 mins 6 secs
2,AG20P1095,Đỗ Thị Ngọc Trinh,thaotrinh0828@gmail.com,ETOP Beginner,"Wednesday, 16 July 2025, 6:57 PM",1 min 17 secs
3,AG20P1095,Đỗ Thị Ngọc Trinh,thaotrinh0828@gmail.com,Pronunciation - Beginner,"Saturday, 19 July 2025, 10:07 AM",1 min 25 secs
4,AG20P1095,Đỗ Thị Ngọc Trinh,thaotrinh0828@gmail.com,ETOP Beginner,"Saturday, 19 July 2025, 10:07 AM",1 min 18 secs


### REGREX PALS_ID


In [4]:
pattern = r"[A-Z]{2}[0-9]{2}[A-Z]{1}[0-9]{4}"

# Kiểm tra PALS_ID có khớp với regex hay không, và loại bỏ các giá trị không khớp
df = df[df["PALS_ID"].str.match(pattern, na=False)]
HTML(df.head(20).to_html())

,PALS_ID,FULLNAME,EMAIL,COURSE,START_TIME,DURATION
2,AG20P1095,Đỗ Thị Ngọc Trinh,thaotrinh0828@gmail.com,ETOP Beginner,"Wednesday, 16 July 2025, 6:57 PM",1 min 17 secs
3,AG20P1095,Đỗ Thị Ngọc Trinh,thaotrinh0828@gmail.com,Pronunciation - Beginner,"Saturday, 19 July 2025, 10:07 AM",1 min 25 secs
4,AG20P1095,Đỗ Thị Ngọc Trinh,thaotrinh0828@gmail.com,ETOP Beginner,"Saturday, 19 July 2025, 10:07 AM",1 min 18 secs
5,AG20P1095,Đỗ Thị Ngọc Trinh,thaotrinh0828@gmail.com,TED Talks - General 1,"Monday, 28 July 2025, 9:17 AM",6 mins 16 secs
6,AG20P1095,Đỗ Thị Ngọc Trinh,thaotrinh0828@gmail.com,English for Starters,"Monday, 28 July 2025, 9:17 AM",5 mins 56 secs
7,AG21P1118,Dương Thị Thu Hồng,thuthanh010621@gmail.com,W341,"Saturday, 5 July 2025, 9:37 AM",3 mins 28 secs
8,AG22P1125,Y Sa A My,ysaamy04@gmail.com,Camper Orientation,"Thursday, 3 July 2025, 6:10 PM",3 mins 27 secs
9,AG22P1125,Y Sa A My,ysaamy04@gmail.com,W218,"Saturday, 5 July 2025, 9:20 AM",2 mins 27 secs
10,AG22P1125,Y Sa A My,ysaamy04@gmail.com,W338,"Saturday, 5 July 2025, 3:19 PM",2 mins 2 secs
11,AG22P1125,Y Sa A My,ysaamy04@gmail.com,W339,"Sunday, 6 July 2025, 7:52 AM",2 mins 31 secs


### FILTER DATA COURSES NAME


#### ACCEPTABLE COURSE NAME


In [5]:
courses_names = [
    " IELTS writing",
    "IELTS speaking",
    "Grammar",
    "English for Beginners",
    "Reading books",
    "Learn English through songs",
    "Learn English through songs - old",
    "Learn English through movies",
    "Ted Talk AI",
    "TED Talks - general 1",
    "TED Talks - general 2",
    "TED Talks - general 3",
    "TED TALKS Career",
    "Ted Talks Career Deck",
    "Self-study IELTS",
    "Self-study TOEIC",
    "Idioms/ collocation",
    "Pronunciation - beginners",
    "Pronunciation - intermediate",
    "Pronunciation - advanced",
    "Learn vocabulary with Quizlet",
    "Sentence Diagramming",
    "How to use AI Chatbots",
    "English Placement Assessment",
    "The library",
    "Tự Học Tiếng Anh 1",
    "Practice for Donor's interview",
    "Report self-study time",
]

#### FILTER COURSES


In [6]:
df = df[df["COURSE"].isin(courses_names)]
HTML(df.head(20).to_html())

,PALS_ID,FULLNAME,EMAIL,COURSE,START_TIME,DURATION
62,AG22P1146,Nguyễn Thị Thanh Trúc,thithanhtuyenle32@gmail.com,Grammar,"Wednesday, 30 July 2025, 11:27 PM",12 mins 34 secs
69,AG22P1157,Trần Thị Hồng Giao,mcvxvx18052021@gmail.com,Learn vocabulary with Quizlet,"Sunday, 27 July 2025, 8:34 AM",15 mins
147,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,"Friday, 11 July 2025, 6:11 PM",1 hour 5 mins
148,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,"Sunday, 13 July 2025, 5:52 PM",2 hours 28 mins
149,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,"Monday, 14 July 2025, 5:53 PM",1 hour 58 mins
150,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,"Tuesday, 15 July 2025, 10:58 AM",1 hour 26 mins
151,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,"Tuesday, 15 July 2025, 6:44 PM",1 hour 39 mins
152,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,"Wednesday, 16 July 2025, 4:53 PM",3 hours 29 mins
153,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,"Thursday, 17 July 2025, 5:14 PM",3 hours 12 mins
154,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,"Friday, 18 July 2025, 7:17 PM",1 hour 6 mins


#### RECHECK COURSE LIST


In [7]:
course_counts = df["COURSE"].value_counts()
print(course_counts)

COURSE
Grammar                          641
Sentence Diagramming             133
Learn vocabulary with Quizlet     73
English Placement Assessment      51
Name: count, dtype: int64


### CONVERT DURATION


In [8]:
# Define the function to convert session duration to minutes
def convert_duration_to_minutes(duration):
    hours = 0
    minutes = 0
    seconds = 0

    # Extract hours, minutes, and seconds using regex
    if "hour" in duration:
        hours_match = re.search(r"(\d+)\s*hour", duration)
        if hours_match:
            hours = int(hours_match.group(1))

    if "min" in duration:
        minutes_match = re.search(r"(\d+)\s*min", duration)
        if minutes_match:
            minutes = int(minutes_match.group(1))

    if "sec" in duration:
        seconds_match = re.search(r"(\d+)\s*sec", duration)
        if seconds_match:
            seconds = int(seconds_match.group(1))

    # Convert to total minutes
    total_minutes = hours + minutes / 60 + seconds / 3600
    return total_minutes

In [9]:
df["DURATION"] = df["DURATION"].apply(convert_duration_to_minutes)
HTML(df.head().to_html())

,PALS_ID,FULLNAME,EMAIL,COURSE,START_TIME,DURATION
62,AG22P1146,Nguyễn Thị Thanh Trúc,thithanhtuyenle32@gmail.com,Grammar,"Wednesday, 30 July 2025, 11:27 PM",0.209444
69,AG22P1157,Trần Thị Hồng Giao,mcvxvx18052021@gmail.com,Learn vocabulary with Quizlet,"Sunday, 27 July 2025, 8:34 AM",0.250000
147,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,"Friday, 11 July 2025, 6:11 PM",1.083333
148,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,"Sunday, 13 July 2025, 5:52 PM",2.466667
149,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,"Monday, 14 July 2025, 5:53 PM",1.966667


### CONVERT START TIME TO UNIX TIMESTAMP


In [10]:
from datetime import datetime


def convert_to_unix_timestamp(date_string):
    try:
        # Phân tích chuỗi ngày giờ
        date_object = datetime.strptime(date_string, "%A, %d %B %Y, %I:%M %p")

        # Chuyển đổi sang Unix timestamp
        unix_timestamp = int(date_object.timestamp())

        return unix_timestamp
    except ValueError:
        return None  # Trả về None nếu chuỗi không hợp lệ


def convert_to_iso8601(date_string):
    try:
        # Phân tích chuỗi ngày giờ
        date_object = datetime.strptime(date_string, "%A, %d %B %Y, %I:%M %p")

        # Chuyển đổi sang định dạng ISO 8601
        iso_format = date_object.isoformat()

        return iso_format
    except ValueError:
        return None  # Trả về None nếu chuỗi không hợp lệ

In [11]:
df["START_TIME"] = df["START_TIME"].apply(convert_to_iso8601)
HTML(df.head(20).to_html())

,PALS_ID,FULLNAME,EMAIL,COURSE,START_TIME,DURATION
62,AG22P1146,Nguyễn Thị Thanh Trúc,thithanhtuyenle32@gmail.com,Grammar,2025-07-30T23:27:00,0.209444
69,AG22P1157,Trần Thị Hồng Giao,mcvxvx18052021@gmail.com,Learn vocabulary with Quizlet,2025-07-27T08:34:00,0.250000
147,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-11T18:11:00,1.083333
148,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-13T17:52:00,2.466667
149,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-14T17:53:00,1.966667
150,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-15T10:58:00,1.433333
151,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-15T18:44:00,1.650000
152,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-16T16:53:00,3.483333
153,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-17T17:14:00,3.200000
154,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-18T19:17:00,1.100000


### CAL END COLUMN


In [12]:
import pandas as pd
from datetime import datetime, timedelta


def add_end_time(df):
    """Thêm cột END vào DataFrame dựa trên START và DURATION."""

    def calculate_end_time(row):
        start_time = datetime.fromisoformat(row["START_TIME"])
        duration_minutes = row["DURATION"]
        duration_timedelta = timedelta(hours=duration_minutes)
        end_time = start_time + duration_timedelta
        return end_time.isoformat()

    df["END_TIME"] = df.apply(calculate_end_time, axis=1)
    return df

In [13]:
df = add_end_time(df)
HTML(df.head(20).to_html())

,PALS_ID,FULLNAME,EMAIL,COURSE,START_TIME,DURATION,END_TIME
62,AG22P1146,Nguyễn Thị Thanh Trúc,thithanhtuyenle32@gmail.com,Grammar,2025-07-30T23:27:00,0.209444,2025-07-30T23:39:34
69,AG22P1157,Trần Thị Hồng Giao,mcvxvx18052021@gmail.com,Learn vocabulary with Quizlet,2025-07-27T08:34:00,0.250000,2025-07-27T08:49:00
147,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-11T18:11:00,1.083333,2025-07-11T19:16:00
148,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-13T17:52:00,2.466667,2025-07-13T20:20:00
149,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-14T17:53:00,1.966667,2025-07-14T19:51:00
150,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-15T10:58:00,1.433333,2025-07-15T12:24:00
151,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-15T18:44:00,1.650000,2025-07-15T20:23:00
152,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-16T16:53:00,3.483333,2025-07-16T20:22:00
153,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-17T17:14:00,3.200000,2025-07-17T20:26:00
154,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-18T19:17:00,1.100000,2025-07-18T20:23:00


## MAIN CALCULATE


### CONVERT ISO 8061 => DATETIME OBJECT


In [14]:
df["START_TIME"] = pd.to_datetime(df["START_TIME"])
df["END_TIME"] = pd.to_datetime(df["END_TIME"])

In [15]:
HTML(df.head(20).to_html())

,PALS_ID,FULLNAME,EMAIL,COURSE,START_TIME,DURATION,END_TIME
62,AG22P1146,Nguyễn Thị Thanh Trúc,thithanhtuyenle32@gmail.com,Grammar,2025-07-30 23:27:00,0.209444,2025-07-30 23:39:34
69,AG22P1157,Trần Thị Hồng Giao,mcvxvx18052021@gmail.com,Learn vocabulary with Quizlet,2025-07-27 08:34:00,0.250000,2025-07-27 08:49:00
147,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-11 18:11:00,1.083333,2025-07-11 19:16:00
148,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-13 17:52:00,2.466667,2025-07-13 20:20:00
149,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-14 17:53:00,1.966667,2025-07-14 19:51:00
150,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-15 10:58:00,1.433333,2025-07-15 12:24:00
151,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-15 18:44:00,1.650000,2025-07-15 20:23:00
152,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-16 16:53:00,3.483333,2025-07-16 20:22:00
153,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-17 17:14:00,3.200000,2025-07-17 20:26:00
154,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-18 19:17:00,1.100000,2025-07-18 20:23:00


### SOLVE CONFLICT TIME


#### CHECK CONFLICT FUNCTION


In [16]:
def check_conflict(t1_start, t1_end, t2_start, t2_end):
    """Kiểm tra xem hai khoảng thời gian có xung đột hay không."""
    return t1_start < t2_end and t2_start < t1_end


def check_conflict_with_15min_gap(t1_start, t1_end, t2_start, t2_end):
    """Kiểm tra xung đột với khoảng cách 15 phút."""
    return max(t1_start, t2_start) < min(t1_end, t2_end) - timedelta(minutes=15)

In [17]:
t1_start = pd.to_datetime("2025-01-21T20:19:00")
t1_end = pd.to_datetime("2025-01-21T20:42:00")
t2_start = pd.to_datetime("2025-01-23T18:29:00")
t2_end = pd.to_datetime("2025-01-23T21:26:00")
print(
    f"Xung đột: {check_conflict(t1_start, t1_end, t2_start, t2_end)}"
)  # Kết quả: True
print(
    f"Xung đột (15 phút): {check_conflict_with_15min_gap(t1_start, t1_end, t2_start, t2_end)}"
)  # Kết quả: True

Xung đột: False
Xung đột (15 phút): False


#### GROUP FOLLOW PALS_ID & CHECK CONFLICT


In [18]:
df["CONFLICT"] = False  # Thêm cột đánh dấu xung đột
conflict_pals_ids = []  # Danh sách PALS_ID bị xung đột
for pals_id, group in df.groupby("PALS_ID"):
    # group = group.sort_values(by='START_TIME')  # Sắp xếp theo thời gian bắt đầu
    for i in range(len(group)):
        for j in range(i + 1, len(group)):
            if check_conflict_with_15min_gap(
                group.iloc[i]["START_TIME"],
                group.iloc[i]["END_TIME"],
                group.iloc[j]["START_TIME"],
                group.iloc[j]["END_TIME"],
            ):

                # print log ở đây
                # print(f"Xung đột giữa {group.iloc[i]['PALS_ID']} (index {group.index[i]}) và {group.iloc[j]['PALS_ID']} (index {group.index[j]})")
                df.loc[group.index[i], "CONFLICT"] = True
                df.loc[group.index[j], "CONFLICT"] = True
                if pals_id not in conflict_pals_ids:
                    conflict_pals_ids.append(pals_id)
HTML(df.head(20).to_html())

,PALS_ID,FULLNAME,EMAIL,COURSE,START_TIME,DURATION,END_TIME,CONFLICT
62,AG22P1146,Nguyễn Thị Thanh Trúc,thithanhtuyenle32@gmail.com,Grammar,2025-07-30 23:27:00,0.209444,2025-07-30 23:39:34,False
69,AG22P1157,Trần Thị Hồng Giao,mcvxvx18052021@gmail.com,Learn vocabulary with Quizlet,2025-07-27 08:34:00,0.250000,2025-07-27 08:49:00,False
147,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-11 18:11:00,1.083333,2025-07-11 19:16:00,False
148,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-13 17:52:00,2.466667,2025-07-13 20:20:00,False
149,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-14 17:53:00,1.966667,2025-07-14 19:51:00,False
150,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-15 10:58:00,1.433333,2025-07-15 12:24:00,False
151,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-15 18:44:00,1.650000,2025-07-15 20:23:00,False
152,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-16 16:53:00,3.483333,2025-07-16 20:22:00,False
153,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-17 17:14:00,3.200000,2025-07-17 20:26:00,False
154,AG23P1187,Nguyễn Thị Bảo Châu,nguyenchau92132506@gmail.com,Grammar,2025-07-18 19:17:00,1.100000,2025-07-18 20:23:00,False


### HIGHLIGHT CONFLICT DATA


In [19]:
def highlight_conflicts(s):
    if s["CONFLICT"]:
        return ["background-color: yellow"] * len(s)
    return [""] * len(s)


styled_df = df.style.apply(highlight_conflicts, axis=1)

## SAVE FILE


In [20]:
from openpyxl import load_workbook

# Lưu DataFrame thành file Excel tạm thời
styled_df.to_excel("./export/temp.xlsx", index=False)

# Load file Excel tạm thời bằng openpyxl
workbook = load_workbook("./export/temp.xlsx")
sheet = workbook.active

# Lưu file Excel với highlight
workbook.save("./export/output_highlighted.xlsx")

# Xóa file Excel tạm thời
import os

os.remove("./export/temp.xlsx")

## SPLIT CONFLICT TIME


### 1. NO CONFLICT DATA


In [21]:
df_valid = df[df["CONFLICT"] == False]

HTML(df_valid.head(20).to_html())
df_valid.to_excel("./export/valid_data.xlsx")
print("SAVE FILE SUCCESSFULLY!")

SAVE FILE SUCCESSFULLY!


### 2. CONFLICT DATA


In [22]:
df_invalid = df[df["CONFLICT"] == True]

HTML(df_invalid.head(20).to_html())
df_invalid.to_excel("./export/invalid_data.xlsx")
print("SAVE FILE SUCCESSFULLY!")

SAVE FILE SUCCESSFULLY!
